# Using EcoFOCIpy to process raw field data

## Mooring / Timeseries Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- Parse data from raw files into pandas dataframe
- output initial files (pandas->csv) **ERDDAP NRT** when no meta data is added

Convert to xarray dataframe for all following work *(working or final data level):
- TODO: Add metadata from instrument yaml files and/or header info
- ingest metadata from deployment/recovery records or cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:


## Example below is for RCM-4/5/7/9/11 Processing pipeline which was developed by D.Pashinski it is different than the RCM-SG (SeaGuard).

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

In [7]:
import yaml

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import EcoFOCIpy.io.rcm_parser as rcm_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config
import EcoFOCIpy.math.geotools as geotools
import EcoFOCIpy.plots.TimeSeriesStickPlot as TSSP

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [8]:
sample_data_dir = '/Users/bell/Programs/EcoFOCIpy/'
user_data_dir = '/Users/bell/ecoraid/2022/Moorings/22bs2c/'

In [9]:
###############################################################
# edit to point to {instrument sepcific} raw datafile 
datafile = user_data_dir+'rawconverted/rcm/22bs2c_rcm9_sn651.xlsx'
instrument = 'RCM9 869'
mooring_meta_file = user_data_dir+'logs/22BS-2C.yaml'
inst_meta_file = sample_data_dir+'staticdata/instr_metaconfig/rcm_cf.yaml'
inst_shortname = 'rcm'
###############################################################

#init and load data
rcm9_wop = rcm_parser.rcm_excel()
(rcm9_wop_data) = rcm9_wop.parse_excel(filename=datafile, datetime_index=False)

In [10]:
rcm9_wop_data.sample(5)

record    speed  direction mag  temperature  \
date_time                                                                 
2022-12-06 21:00:00           2228  19.6511       148.3752     3.860873   
2022-10-14 07:00:00            942  27.5702       358.6320     7.700748   
2023-01-21 09:59:59.999000    3321  61.5930       253.5036     3.721065   
2022-12-16 02:59:59.999000    2450  10.2655       151.8912     3.767671   
2022-12-23 00:59:59.999000    2616  28.4501        28.8312     3.348047   

                            salinity  pressure  east true  north true  \
date_time                                                               
2022-12-06 21:00:00              NaN  13.79919   7.491020  -18.167288   
2022-10-14 07:00:00              NaN  13.79919   3.764921   27.311926   
2023-01-21 09:59:59.999000       NaN  12.37499 -61.096523   -7.804644   
2022-12-16 02:59:59.999000       NaN  13.79919   3.323835   -9.712498   
2022-12-23 00:59:59.999000       NaN  13.08709  17.534367   22.404334   

                            conductivity  turbidity  o2saturation  \
date_time                                                           
2022-12-06 21:00:00                  NaN        NaN           NaN   
2022-10-14 07:00:00                  NaN        NaN           NaN   
2023-01-21 09:59:59.999000           NaN        NaN           NaN   
2022-12-16 02:59:59.999000           NaN        NaN           NaN   
2022-12-23 00:59:59.999000           NaN        NaN           NaN   

                            o2concentration  Unnamed: 13  O2corrected  \
date_time                                                               
2022-12-06 21:00:00                     NaN          NaN          NaN   
2022-10-14 07:00:00                     NaN          NaN          NaN   
2023-01-21 09:59:59.999000              NaN          NaN          NaN   
2022-12-16 02:59:59.999000              NaN          NaN          NaN   
2022-12-23 00:59:59.999000              NaN          NaN          NaN   

                            o2_saturation  o2(uM/kg)  
date_time                                             
2022-12-06 21:00:00                   NaN        NaN  
2022-10-14 07:00:00                   NaN        NaN  
2023-01-21 09:59:59.999000            NaN        NaN  
2022-12-16 02:59:59.999000            NaN        NaN  
2022-12-23 00:59:59.999000            NaN        NaN

## Oxygen and other ancillary sensor adjusments

Oxygen units should be adjusted and salinity corrections as we initiallize oxygen sensors as if in a 0 PSU environment.  Without a cond cell on board, this needs a secondary instrument.  See QC section below

## Time properties

Its unusual that our clocks drift to the point of concern for our instruments (if an instrument is off by 3 minutes but only sampling hourly... regridding that data will result in minimal changes).  However, there are a few time oriented modifications that may need to be made.

The can be classified into two categories:
+ interpolate: these change the parameter values in accordance with the time edits
    - linear interpolation is most common
    - averaging of data and rebinning/resampling is also common (this needs to have the "time lable" thought out...)
    - decimating is less common but does not impact the max/min values
+ shift: these do not alter the measurements, just the timestamps they are associated with
    - the round function will work well to correct small time errors/drifts **common**
    - dropping extra precision on time (if you want hourly measurements, just remove all minute/second info... could cause large errors if rounding would have been more appropriate)

It is very easy to use pandas interplation and resample methods on the dataframe as is.  A few steps are suggested below:
- parse out on-deck (predeployment and recovery) data.  This can be done via pandas or xarray but requires the mooring metadata to have been read in.  See future steps below.
- even if the sample frequency is set to the desired measurement frequency, it would be good to perform a quick regridding as an assurance task
- FOCI data is usualy 1min, 10min, 1hr - and the 1min data is a fairly new (sbe56) data stream
    + subsampling high frequency data to lower frequency is easy via df.resample().mean() but it will label the new datapoint per default instructions.  The default is to label it with the left boundary of the bin.
    + you may want to take the median instead of the mean for noisy data (fluorometer) , occasionally decimating may be more appropriate if you want to downsize the dataset size but not smear features
    + shifting times can be a bit more involved.  There are two primary ways to do it, interpolate or shift (round)
        - to interpolate, you will need to upsample your data to a higher frequency which will generate missing values, then interpolate (with a maximum gap size), then decimate.  This always has the artifact of smoothing data and decreasing the min/max values. **common on microcats and other 10min datasets**
        - shifting usually just involves droping extra time "digits", if you want hourly, you could just drop the trailing minutes assuming you are just off the hour (8:05 -> 8:00) or you can round to the nearest time unit but niether of these changes the data value, just the time associated with it. **common on seacats and other hourly datasets**
        - you may also be able to *shift* using the pandas datetime round function and specifing the desired frequency.
    + I suggest if no change is needed... df.index.round(freq=*'your native sample freq'*)
    
    
***Half Hour Sample - rounded/truncated***

In [17]:
import pandas as pd
rcm9_wop_data.index = pd.DatetimeIndex(rcm9_wop_data.index)

In [18]:
rcm9_wop_data.index = rcm9_wop_data.index.round(freq='1H')
rcm9_wop_data

record    speed  direction mag  temperature  salinity  \
date_time                                                                    
2022-09-05 02:00:00       1  29.0367        92.1192     9.991231       NaN   
2022-09-05 03:00:00       2  25.5171       106.5348    10.038160       NaN   
2022-09-05 04:00:00       3  20.5310       116.7312    10.132047       NaN   
2022-09-05 05:00:00       4  13.7851       132.5532    10.108572       NaN   
2022-09-05 06:00:00       5   6.7459       172.6356    10.085099       NaN   
...                     ...      ...            ...          ...       ...   
2023-05-05 19:00:00    5826  18.1846       236.9784    21.687630       NaN   
2023-05-05 20:00:00    5827  23.7573       157.1652    21.687630       NaN   
2023-05-05 21:00:00    5828  18.7712        29.8860    21.687630       NaN   
2023-05-05 22:00:00    5829  10.5588       137.1240    21.325178       NaN   
2023-05-05 23:00:00    5830  21.4109       203.9280    14.431477       NaN   

                     pressure  east true  north true  conductivity  turbidity  \
date_time                                                                       
2022-09-05 02:00:00  13.79919  28.470243   -5.707469           NaN        NaN   
2022-09-05 03:00:00  13.79919  22.982917  -11.086385           NaN        NaN   
2022-09-05 04:00:00  13.79919  16.620895  -12.052710           NaN        NaN   
2022-09-05 05:00:00  13.79919   8.530512  -10.828636           NaN        NaN   
2022-09-05 06:00:00  13.79919  -0.218048   -6.742375           NaN        NaN   
...                       ...        ...         ...           ...        ...   
2023-05-05 19:00:00   0.26929 -16.637548   -7.339733           NaN        NaN   
2023-05-05 20:00:00   0.26929   5.593636  -23.089403           NaN        NaN   
2023-05-05 21:00:00   0.26929  11.839228   14.566764           NaN        NaN   
2023-05-05 22:00:00   0.26929   5.852250   -8.788597           NaN        NaN   
2023-05-05 23:00:00   0.26929 -11.706524  -17.927184           NaN        NaN   

                     o2saturation  o2concentration  Unnamed: 13  O2corrected  \
date_time                                                                      
2022-09-05 02:00:00           NaN              NaN          NaN          NaN   
2022-09-05 03:00:00           NaN              NaN          NaN          NaN   
2022-09-05 04:00:00           NaN              NaN          NaN          NaN   
2022-09-05 05:00:00           NaN              NaN          NaN          NaN   
2022-09-05 06:00:00           NaN              NaN          NaN          NaN   
...                           ...              ...          ...          ...   
2023-05-05 19:00:00           NaN              NaN          NaN          NaN   
2023-05-05 20:00:00           NaN              NaN          NaN          NaN   
2023-05-05 21:00:00           NaN              NaN          NaN          NaN   
2023-05-05 22:00:00           NaN              NaN          NaN          NaN   
2023-05-05 23:00:00           NaN              NaN          NaN          NaN   

                     o2_saturation  o2(uM/kg)  
date_time                                      
2022-09-05 02:00:00            NaN        NaN  
2022-09-05 03:00:00            NaN        NaN  
2022-09-05 04:00:00            NaN        NaN  
2022-09-05 05:00:00            NaN        NaN  
2022-09-05 06:00:00            NaN        NaN  
...                            ...        ...  
2023-05-05 19:00:00            NaN        NaN  
2023-05-05 20:00:00            NaN        NaN  
2023-05-05 21:00:00            NaN        NaN  
2023-05-05 22:00:00            NaN        NaN  
2023-05-05 23:00:00            NaN        NaN  

[5830 rows x 16 columns]

In [19]:
rcm9_wop_data.plot(figsize=(16,2),legend=False)

<Axes: xlabel='date_time'>

## Add Deployment meta information

Two methods are available (if comming from python2 world - ordereddict was important... in py38 a dictionary is inherently ordered)

In [20]:
#just a dictionary of dictionaries - simple
with open(mooring_meta_file) as file:
    mooring_config = yaml.full_load(file)

In [21]:
#some parsing/cleaning for functions
latlon_dec = geotools.latlon_convert(mooring_config['Deployment']['DeploymentLatitude'],
                        mooring_config['Deployment']['DeploymentLongitude'])
dep_date = mooring_config['Deployment']['DeploymentDateTimeGMT'].date()

In [22]:
mooring_config['Instrumentation'][instrument]

{'InstType': 'RCM9',
 'SerialNo': '869',
 'DesignedDepth': 15.0,
 'ActualDepth': 15.0,
 'PreDeploymentNotes': 'Alk Batts, Regular DSU, 60 min',
 'PostDeploymentNotes': '',
 'Deployed': 'y',
 'Recovered': ''}

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [23]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature': {'epic_key': 'T_20',
  'generic_name': 'temp',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'pressure': {'epic_key': 'P_1',
  'generic_name': 'pres',
  'units': 'decibar',
  'long_name': 'Sea water pressure, equals 0

In [24]:
#rcm9 data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#TODO: Manage ancillary instrumentation
rcm9_wop_data = rcm9_wop_data.rename(columns={'Temperature(DegC)':'temperature',
                        'o2(uM/kg)':'oxy_conc_umkg', #unit conversions all done in excel? be careful here
                        'o2_saturation':'oxy_percentsat',
                        'East(cm/s)':'u_curr_comp',
                        'North(cm/s)':'v_curr_comp',
                        'speed':'current_speed',
                        'direction mag':'current_direction'
                        })
rcm9_wop_data['oxy_conc'] = np.nan

rcm9_wop_data.sample()


record  current_speed  current_direction  temperature  \
date_time                                                                    
2022-11-01 05:00:00    1372        12.0253           248.5812      5.79237   

                     salinity  pressure  east true  north true  conductivity  \
date_time                                                                      
2022-11-01 05:00:00       NaN  13.08709 -11.753626   -2.541675           NaN   

                     turbidity  o2saturation  o2concentration  Unnamed: 13  \
date_time                                                                    
2022-11-01 05:00:00        NaN           NaN              NaN          NaN   

                     O2corrected  oxy_percentsat  oxy_conc_umkg  oxy_conc  
date_time                                                                  
2022-11-01 05:00:00          NaN             NaN            NaN       NaN

## Magnetic Declination Correction

In [25]:
#done in excel file
dec_corr=21.15

## Calculate U,V for RCM 7/9/11

In [26]:
rcm9_wop_data['u_curr_comp'] = rcm9_wop_data['current_speed']*np.sin(np.deg2rad(rcm9_wop_data['current_direction']))
rcm9_wop_data['v_curr_comp'] = rcm9_wop_data['current_speed']*np.cos(np.deg2rad(rcm9_wop_data['current_direction']))

## Visualization of Mag Dec Corrected Current, Temperture and Oxy (if available) or any other parameter

In [28]:
stickplot = TSSP.Timeseries1dStickPlot()
fig,ax = stickplot.plot(rcm9_wop_data.index,udata=rcm9_wop_data['u_curr_comp'],vdata=rcm9_wop_data['v_curr_comp'], rotate=0)

plt_sub = rcm9_wop_data[['temperature','oxy_conc_umkg','oxy_percentsat']]

fig, ax = plt.subplots(nrows=len(plt_sub.columns), sharex=True, figsize=(16,8))
for count,parameter in enumerate(plt_sub.columns):
    plt_sub[parameter].plot(ax=ax[count],label=parameter)
    ax[count].legend(loc='upper left')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/plots/TimeSeriesStickPlot.py:119: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), visible=True)


In [29]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
rcm9_wop_nc = ncCFsave.EcoFOCI_CFnc(df=rcm9_wop_data, 
                                instrument_yaml=inst_config, 
                                operation_yaml=mooring_config,
                                operation_type='mooring', 
                                instrument_id=instrument, 
                                inst_shortname=inst_shortname)
rcm9_wop_nc

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [30]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
rcm9_wop_nc.expand_dimensions()

In [31]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
rcm9_wop_nc.variable_meta_data(variable_keys=list(rcm9_wop_data.columns.values),drop_missing=True)
rcm9_wop_nc.temporal_geospatioal_meta_data(depth='designed')
#adding dimension meta needs to come after updating the dimension values... BUG?
rcm9_wop_nc.dimension_meta_data(variable_keys=['depth','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [32]:
#add global attributes
rcm9_wop_nc.deployment_meta_add()
rcm9_wop_nc.get_xdf()

#add instituitonal global attributes
rcm9_wop_nc.institution_meta_add()

#add creation date/time - provenance data
rcm9_wop_nc.provinance_meta_add()

#provide intial qc status field
rcm9_wop_nc.qc_status(qc_status='unknown')

#declination correction as part of history
rcm9_wop_nc.history(f'Declination Correction = {dec_corr}')

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [33]:
# combine trim (not mandatory) and filename together (saves to test.nc without name)

depth = str(int(mooring_config['Instrumentation'][instrument]['ActualDepth'])).zfill(4)
# mooring_yaml['Instrumentation'][self.instrument_id]['DesignedDepth'])).zfill(4) #<-- alternative
filename = "".join(mooring_config['MooringID'].split('-')).lower()+'_'+inst_shortname+'_'+depth+'m.nc'
rcm9_wop_nc.xarray2netcdf_save(xdf = rcm9_wop_nc.autotrim_time(),
                           filename=filename,format="NETCDF3_CLASSIC")

#csvoutput if necessary
(rcm9_wop_nc.get_xdf().to_dataframe()).to_csv(filename.replace('nc','csv'))

(rcm9_wop_nc.autotrim_time().to_dataframe()).to_csv(filename.replace('nc','trimmed.csv'))

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'hours' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


In [34]:
rcm9_wop_nc.get_xdf()

<xarray.Dataset>
Dimensions:            (time: 5830, depth: 1, latitude: 1, longitude: 1)
Coordinates:
  * time               (time) datetime64[ns] 2022-09-05T02:00:00 ... 2023-05-...
  * latitude           (latitude) float64 56.87
  * longitude          (longitude) float64 -164.1
  * depth              (depth) float64 15.0
Data variables:
    current_speed      (time, depth, latitude, longitude) float64 29.04 ... 2...
    current_direction  (time, depth, latitude, longitude) float64 92.12 ... 2...
    temperature        (time, depth, latitude, longitude) float64 9.991 ... 1...
    salinity           (time, depth, latitude, longitude) float64 nan ... nan
    pressure           (time, depth, latitude, longitude) float64 13.8 ... 0....
    oxy_percentsat     (time, depth, latitude, longitude) float64 nan ... nan
    oxy_conc_umkg      (time, depth, latitude, longitude) float64 nan ... nan
    oxy_conc           (time, depth, latitude, longitude) float64 nan ... nan
    u_curr_comp        (time, depth, latitude, longitude) float64 29.02 ... -...
    v_curr_comp        (time, depth, latitude, longitude) float64 -1.074 ... ...
Attributes: (12/13)
    Latitude_DegMMddW:                      56 52.456 N
    Longitude_DegMMddN:                     164 03.954 W
    MooringID:                              22BS-2C
    platform_deployment_date:               2022-09-04T20:40:00Z
    platform_deployment_cruise_name:        AQ2201
    platform_recovery_date:                 2023-04-28T00:00:00Z
    ...                                     ...
    platform_deployment_recovery_comments:  Remove all caps, start all fluoro...
    WaterDepth:                             73.0
    date_created:                           2024-12-26T14:04:29Z
    date_modified:                          
    QC_indicator:                           unknown
    history:                                Declination Correction = 21.15\nT...